In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

df_fake = pd.read_csv('Fake.csv', header=None, names=['titulo', 'mensagem', 'tipo' ,'data'])
df_fake['target'] = 0
df_true = pd.read_csv('True.csv', header=None, names=['titulo', 'mensagem', 'tipo' ,'data'])
df_true['target'] = 1
df = pd.concat([df_fake, df_true], ignore_index=True)

# Preprocessamento: vetorização tf-idf e escalação
vectorizer = TfidfVectorizer(max_features=18000, min_df=2 ,stop_words='english', dtype=np.float32)
tfidf = vectorizer.fit_transform(df['mensagem'])
x_tfidf = pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out())

scaler = StandardScaler()
x_tfidf_scaled = scaler.fit_transform(x_tfidf)

# Separação dos dados para Validação Cruzada 
kfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=7)
indice_fold = 0

# Geração de modelos e suas Estatísticas
status_modelos = []
metricas_dict = {
    'accuracy': accuracy_score, 
    'precision': precision_score,
    'recall': recall_score,
    'roc_curve': roc_auc_score,
    'f1_score': f1_score,
    'sensibilidade': None,
    'especifidade': None
    }
ks = [ k for k in range(1,8,1)]

for indices_treino, indices_teste in kfolds.split(x_tfidf_scaled, df['target']):
    x_treino, x_teste = x_tfidf_scaled[indices_treino], x_tfidf_scaled[indices_teste]
    y_treino, y_teste = df['target'].iloc[indices_treino],df['target'].iloc[indices_teste] 
    for k in ks:

        modeloKNN = KNeighborsClassifier(n_neighbors=k).fit(x_treino, y_treino)
        y_predict = modeloKNN.predict(x_teste)
        tn, fp, fn, tp  = confusion_matrix(y_teste, y_predict).ravel()
        for metrica in ['accuracy', 'precision', 'recall', 'roc_curve', 'f1_score', 'sensibilidade', 'especifidade']:
            if metrica != 'sensibilidade' and metrica != 'especifidade':
                metrica_calc = metricas_dict[metrica](y_teste, y_predict)
            elif metrica == 'sensibilidade':
                metrica_calc = tp / (tp + fn)
            else:
                metrica_calc = tn / (tn + fp)
            status_modelos.append({'k': k, 'fold': indice_fold, 'metrica': metrica, 'valor': metrica_calc})
    
    indice_fold += 1


# Rankeando as métricas fazendo utilizando todos os folds para cada K
metricas_modelos = pd.DataFrame().from_records(status_modelos)
modelos = metricas_modelos.groupby(['k', 'metrica']).agg({'valor': ['mean', 'max','min']}).sort_values(by=['metrica', ('valor', 'max'), ('valor', 'mean'), ('valor', 'min')], ascending=[True, False, False, False])
modelos

valor                    
                     mean       max       min
k metrica                                    
1 accuracy       0.564928  0.566280  0.562540
2 accuracy       0.518203  0.518889  0.517701
3 accuracy       0.518150  0.518889  0.517701
4 accuracy       0.507620  0.508413  0.507223
5 accuracy       0.507620  0.508413  0.507223
6 accuracy       0.504709  0.504921  0.504366
7 accuracy       0.504709  0.504921  0.504366
6 especifidade   1.000000  1.000000  1.000000
7 especifidade   1.000000  1.000000  1.000000
4 especifidade   0.999895  1.000000  0.999684
5 especifidade   0.999895  1.000000  0.999684
2 especifidade   0.999578  1.000000  0.999367
3 especifidade   0.999473  1.000000  0.999051
1 especifidade   0.995043  0.995887  0.994304
  f1_score       0.231994  0.237298  0.223224
2 f1_score       0.064904  0.067671  0.062346
3 f1_score       0.064897  0.067671  0.062346
4 f1_score       0.023710  0.026407  0.022054
5 f1_score       0.023710  0.026407  0.022054
6 f1_score       0.012033  0.012662  0.010773
7 f1_score       0.012033  0.012662  0.010773
6 precision      1.000000  1.000000  1.000000
7 precision      1.000000  1.000000  1.000000
4 precision      0.990991  1.000000  0.972973
5 precision      0.990991  1.000000  0.972973
2 precision      0.987817  1.000000  0.981308
3 precision      0.984788  1.000000  0.972222
1 precision      0.963657  0.968215  0.959368
  recall         0.131889  0.135393  0.126155
2 recall         0.033556  0.035043  0.032176
3 recall         0.033556  0.035043  0.032176
4 recall         0.012000  0.013380  0.011150
5 recall         0.012000  0.013380  0.011150
6 recall         0.006053  0.006371  0.005416
7 recall         0.006053  0.006371  0.005416
1 roc_curve      0.563466  0.564849  0.561021
2 roc_curve      0.516567  0.517205  0.516088
3 roc_curve      0.516514  0.517205  0.516088
4 roc_curve      0.505947  0.506690  0.505575
5 roc_curve      0.505947  0.506690  0.505575
6 roc_curve      0.503026  0.503186  0.502708
7 roc_curve      0.503026  0.503186  0.502708
1 sensibilidade  0.131889  0.135393  0.126155
2 sensibilidade  0.033556  0.035043  0.032176
3 sensibilidade  0.033556  0.035043  0.032176
4 sensibilidade  0.012000  0.013380  0.011150
5 sensibilidade  0.012000  0.013380  0.011150
6 sensibilidade  0.006053  0.006371  0.005416
7 sensibilidade  0.006053  0.006371  0.005416